In [1]:
# 2022-09-02 16:23 Seoul

# --- import dataset ---
from utils.dataloader import mel_dataset
from utils.losses import *
from torch.utils.data import DataLoader, random_split

# --- import model ---
from model.supervised_model import *
from model.Conv1d_model import Conv1d_VAE
# from model.Conv1d_model import Encoder as Encoder1d

from model.Conv2d_model import Conv2d_VAE
from model.Conv2d_model import Encoder

# --- import framework ---
import flax 
import flax.linen as nn
from flax.training import train_state
import jax
import numpy as np
import jax.numpy as jnp
import optax

import cloudpickle
import argparse
from tqdm import tqdm
import os
import wandb
import matplotlib.pyplot as plt

In [2]:
# --- collate batch for dataloader ---
def collate_batch(batch):
    x_train = [x for x, _ in batch]
    y_train = [y for _, y in batch]                  
        
    return np.array(x_train), np.array(y_train)



# --- define init state ---
def init_state(model, x_shape, key, lr) -> train_state.TrainState:
    params = model.init({'params': key}, jnp.ones(x_shape), key)
    optimizer = optax.adam(learning_rate=lr)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        tx=optimizer,
        params=params)




# --- define train_step ---
@jax.jit
def train_step(state, x, z_rng):    
    
    def loss_fn(params):
        recon_x, mean, logvar = model.apply(params, x, z_rng)
        kld_loss = kl_divergence(mean, logvar).mean()
        mse_loss = ((recon_x - x)**2).mean()
        loss = mse_loss + kld_loss
        return loss
    
    grad_fn = jax.value_and_grad(loss_fn)
    loss, grads = grad_fn(state.params)
    
    return state.apply_gradients(grads=grads), loss

@jax.jit
def linear_train_step(encoder, 
                      enc_state, 
                      enc_batch, 
                      linear_state, 
                      x, y):    
    
    latent = encoder.apply({'params':enc_state, 'batch_stats':enc_batch}, x)
    
    def loss_fn(params):
        logits = linear_evaluation().apply(params, latent)
        loss = jnp.mean(optax.softmax_cross_entropy(logits, y))
        return loss, logits
    
    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(linear_state.params)
    accuracy = jnp.mean(jnp.argmax(logits, -1) == jnp.argmax(labels, -1))
    
    return state.apply_gradients(grads=grads), loss



# --- define eval step ---
@jax.jit
def eval_step(state, x, z_rng):
    
    recon_x, mean, logvar = model.apply(state.params, x, z_rng)
    kld_loss = kl_divergence(mean, logvar).mean()
    mse_loss = ((recon_x - x)**2).mean()
    loss = mse_loss + kld_loss
    
    return recon_x, loss, mse_loss, kld_loss

@jax.jit
def linear_eval_step(state, x, y):
    
    logits = linear_evaluation().apply(state.params, x)
    loss = jnp.mean(optax.softmax_cross_entropy(logits, y))
    accuracy = jnp.mean(jnp.argmax(logits, -1) == jnp.argmax(labels, -1))
    
    return loss, accuracy



In [3]:
batch_size = 16
lr = 0.0001
dilation = True


model = Conv2d_VAE(dilation=dilation)
rng = jax.random.PRNGKey(303)


# ---Load dataset---
dataset_dir = os.path.join(os.path.expanduser('~'),'dataset')            

print("Loading dataset...")    
data = mel_dataset(dataset_dir)
print(f'Loaded data : {len(data)}\n')

dataset_size = len(data)
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size

train_dataset, test_dataset = random_split(data, [train_size, test_size])


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=int(batch_size/4), shuffle=True, num_workers=0, collate_fn=collate_batch)


print('Data load complete!\n')
print(nn.tabulate(model, rngs={'params': rng})(next(iter(train_dataloader))[0], rng))



# ---initializing model---
print("Initializing model....")
state = init_state(model, 
                   next(iter(train_dataloader))[0].shape, 
                   rng, 
                   lr)

print("Initialize complete!!\n")



Loading dataset...
Load song_meta.json...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 707989/707989 [00:00<00:00, 815762.83it/s]


Load complete!

Load file list...


111it [00:07, 14.09it/s]


Loaded data : 93389

Data load complete!



                                     Conv2d_VAE Summary                                      
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ path                  ┃ outputs               ┃ batch_stats        ┃ params               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ Inputs                │ - float32[16,48,1876] │                    │                      │
│                       │ - uint32[2]           │                    │                      │
├───────────────────────┼───────────────────────┼────────────────────┼──────────────────────┤
│ decoder/BatchNorm_0   │ float32[16,12,469,32] │ mean: float32[32]  │ bias: float32[32]    │
│                       │                       │ var: float32[32]   │ scale: float32[32]   │
│                       │                       │                    │                      │
│                       │                       │ 64 (256 B)         │ 64 (256 B)           │
├───────────────────────┼───────────────────────┼────────────────────┼──────────────────────┤
│ decoder/BatchNorm_1   │ float32[16,12,469,64] │ mean: float32[64]  │ bias: float32[64]    │
│                       │                       │ var: float32[64]   │ scale: float32[64]   │
│                       │                       │                    │                      │
│                       │                       │ 128 (512 B)        │ 128 (512 B)          │
├───────────────────────┼───────────────────────┼────────────────────┼──────────────────────┤
│ decoder/BatchNorm_2   │ float32[16,24,938,12… │ mean: float32[128] │ bias: float32[128]   │
│                       │                       │ var: float32[128]  │ scale: float32[128]  │
│                       │                       │                    │                      │
│                       │                       │ 256 (1.0 KB)       │ 256 (1.0 KB)         │
├───────────────────────┼───────────────────────┼────────────────────┼──────────────────────┤
│ decoder/ConvTranspos… │ float32[16,12,469,32] │                    │ bias: float32[32]    │
│                       │                       │                    │ kernel:              │
│                       │                       │                    │ float32[3,3,1,32]    │
│                       │                       │                    │                      │
│                       │                       │                    │ 320 (1.3 KB)         │
├───────────────────────┼───────────────────────┼────────────────────┼──────────────────────┤
│ decoder/ConvTranspos… │ float32[16,12,469,64] │                    │ bias: float32[64]    │
│                       │                       │                    │ kernel:              │
│                       │                       │                    │ float32[3,3,32,64]   │
│                       │                       │                    │                      │
│                       │                       │                    │ 18,496 (74.0 KB)     │
├───────────────────────┼───────────────────────┼────────────────────┼──────────────────────┤
│ decoder/ConvTranspos… │ float32[16,24,938,12… │                    │ bias: float32[128]   │
│                       │                       │                    │ kernel:              │
│                       │                       │                    │ float32[3,3,64,128]  │
│                       │                       │                    │                      │
│                       │                       │                    │ 73,856 (295.4 KB)    │
├───────────────────────┼───────────────────────┼────────────────────┼──────────────────────┤
│ decoder/ConvTranspos… │ float32[16,48,1876,2… │                    │ bias: float32[256]   │
│                       │                       │                    │ kernel:              │
│                       │                       │   




Initializing model....
Initialize complete!!



In [4]:
def linear_init_state(model, x_shape, key, lr) -> train_state.TrainState:
    params = model.init({'params': key}, jnp.ones(x_shape))
    optimizer = optax.adam(learning_rate=lr)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        tx=optimizer,
        params=params)


In [17]:
@jax.jit
def linear_train_step(enc_state, 
                      enc_batch, 
                      linear_state, 
                      x, y):    
    
    latent, _, _ = Encoder().apply({'params':enc_state, 'batch_stats':enc_batch}, x, rng)
    
    def loss_fn(params):
        logits = linear_evaluation().apply(params, latent)
        loss = jnp.mean(optax.softmax_cross_entropy(logits, y))
        return loss, logits
    
    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(linear_state.params)
    accuracy = jnp.mean(jnp.argmax(logits, -1) == jnp.argmax(y, -1))
    
    return state.apply_gradients(grads=grads), loss


In [6]:
linear_state = linear_init_state(linear_evaluation(), (16, 512), rng, lr)

In [7]:
print('Linear evalutaion step.')

enc_state = state.params['params']['encoder']
enc_batch = state.params['batch_stats']['encoder']


Linear evalutaion step.


In [8]:
train_data = iter(train_dataloader)
test_data = iter(test_dataloader)

train_loss_mean = 0
test_loss_mean = 0

In [9]:
x, y = next(train_data)

In [10]:
x = x + 100

In [11]:
latent, mean, log = Encoder().apply({'params':enc_state, 'batch_stats':enc_batch}, x, rng)

In [12]:
x.shape

(16, 48, 1876)

In [13]:
mean.shape

(16, 512)

In [14]:
nn.tabulate(linear_evaluation(), rng)(latent[0])

                   linear_evaluation Summary                   
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path              ┃ outputs      ┃ params                   ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inputs            │ float32[512] │                          │
├───────────────────┼──────────────┼──────────────────────────┤
│ Dense_0           │ float32[512] │ bias: float32[512]       │
│                   │              │ kernel: float32[512,512] │
│                   │              │                          │
│                   │              │ 262,656 (1.1 MB)         │
├───────────────────┼──────────────┼──────────────────────────┤
│ Dense_1           │ float32[30]  │ bias: float32[30]        │
│                   │              │ kernel: float32[512,30]  │
│                   │              │                          │
│                   │              │ 15,390 (61.6 KB)         │
├───────────────────┼──────────────┼──────────────────────────┤
│ linear_evaluation │ float32[30]  │                          │
├───────────────────┼──────────────┼──────────────────────────┤
│                   │        Total │ 278,046 (1.1 MB)         │
└───────────────────┴──────────────┴──────────────────────────┘
                                                               
              Total Parameters: 278,046 (1.1 MB)               

'\n\n'

In [15]:
x.shape

(16, 48, 1876)

In [ ]:
latent = Encoder2d().apply({'params':enc_state, 'batch_stats':enc_batch}, x, rng)

def loss_fn(params):
    logits = linear_evaluation().apply(params, latent)
    loss = jnp.mean(optax.softmax_cross_entropy(logits, y))
    return loss, logits

grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
(loss, logits), grads = grad_fn(linear_state.params)
accuracy = jnp.mean(jnp.argmax(logits, -1) == jnp.argmax(labels, -1))

return state.apply_gradients(grads=grads), loss

In [18]:
linear_state, loss = linear_train_step(
                  enc_state=enc_state, 
                  enc_batch=enc_batch, 
                  linear_state=linear_state, 
                 x=x, y=y)

ValueError: Dict key mismatch; expected keys: ['params']; dict: {'batch_stats': {'decoder': {'BatchNorm_0': {'mean': DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0.], dtype=float32), 'var': DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0.], dtype=float32)}, 'BatchNorm_1': {'mean': DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0.], dtype=float32), 'var': DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 